In [4]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt



ImportError: cannot import name 'axes' from 'matplotlib' (c:\Users\Rasmus\.virtualenvs\Machine-learning-Rasmus-Johansson-C-TUgvqA\lib\site-packages\matplotlib\__init__.py)